In [3]:
#Codigo general
#Si quieres ver como lo programamos, revisa el video donde lo explico
#todo desde cero: https://youtu.be/iX_on3VxZzk

import tensorflow as tf
import numpy as np

celsius = np.array([-40, -10, 0, 8, 15, 22, 38, 25, 57], dtype=float)
fahrenheit = np.array([-40, 14, 32, 46, 59, 72, 100, 77, 134], dtype=float)

oculta1 = tf.keras.layers.Dense(units=5, input_shape=[1])
oculta2 = tf.keras.layers.Dense(units=5)
salida = tf.keras.layers.Dense(units=1)
modelo = tf.keras.Sequential([oculta1, oculta2, salida])

modelo.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss='mean_squared_error'
)

print("Comenzando entrenamiento...")
historial = modelo.fit(celsius, fahrenheit, epochs=300, verbose=False)
print("Modelo entrenado!")

Comenzando entrenamiento...
Modelo entrenado!


In [4]:
resultado = modelo.predict([100.0])
print("Prediccion: 100 grados celsius son " + str(resultado) + " fahrenheit!")

Prediccion: 100 grados celsius son [[211.4601]] fahrenheit!


In [5]:
#Exportar el modelo en formato h5
modelo.save('celsius_a_fahrenheit.h5')

In [18]:
#Crear carpeta donde se colocaran los archivos resultantes
!mkdir carpeta_salida

In [6]:
#Realizar la exportacion a la carpeta de salida
!"C:\Users\fabian\AppData\Roaming\Python\Python310\Scripts\tensorflowjs_converter" --input_format keras celsius_a_fahrenheit.h5 carpeta_salida

2022-04-16 23:45:50.017047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-16 23:45:50.017409: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
# Convert keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
converter.optimization = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("celcius_a_fahrenheit.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\fabian\AppData\Local\Temp\tmpbzin2vk7\assets


1676

In [8]:
# Function: Convert some hex values into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nconst unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable 
    c_str += 'const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i+1) < len(hex_data):
            hex_str += ','
        if (i+1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n' + format(' '.join(hex_array)) + '\n};\n\n'

    #Close out Header guard
    c_str += '#endif // ' + var_name.upper() + '_H'

    return c_str

In [9]:
# Write TFLite model to a C source (or header) file
with open('celcius_a_fahrenheit.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, 'fahrenheit'))